## Julia的代码设计 —— 从 Yao.jl 谈起

Julia的代码设计方式和Python/C++等传统的OOP语言有很大不同，这种很大的不同往往是由于没有 `class` 导致的，我们下面将谈谈怎么发挥Julia的优势而避免一些劣势。

## Julia语言里的面向对象

## 问题

**Yao.jl** 的主要任务之一是对一个量子线路进行建模。我们这里不会主要介绍量子计算，但是这不妨碍我们利用这个场景理解Julia程序设计上的不同。我们大概看一下一个量子线路长什么样子：

![](https://quantumbfs.github.io/Yao.jl/stable/assets/figures/differentiable.png)


![](https://quantumbfs.github.io/Yao.jl/stable/assets/figures/qft.png)

我们先来想想在 OOP 的思路里会怎么为这样一个场景建模

首先定义一个接口类

In [4]:
class BasicBlock:
    """
    接口类
    """
    def __init__(self):
        self.name = "none"

    def apply(self, reg):
        raise NotImplementedError # 强制子类型实现的方法


然后一些不同的门（或者块）将会继承这个接口类，我们首先将所有的门分为两类：

- 能够产生一个矩阵的
- 其他的（例如，测量，队列等）

In [5]:
class MatrixBlock(BasicBlock):
    
    def __init__(self):
        pass
    
    def mat(self):
        print("get matrix")

而对于能够产生矩阵的块，我们还可以细分：

- 组合块（Composite Block）
- 基本块（Primitive Block）

In [13]:
class CompositeBlock(MatrixBlock):
    
    def __init__(self, *args):
        # 一些公共成员
        pass
    
class PrimitiveBlock(MatrixBlock):
    
    def __init__(self, *args):
        pass
    
class XGate(PrimitiveBlock):
    
    def apply(self, reg):
        print("apply X gate")

        
class Chain(CompositeBlock):
    
    def __init__(self, *args):
        self.blks = args
        
    def apply(self, reg):
        print("apply a chain")

而这个时候我们又要求上面的这些块的矩阵形式都能够被一个服务器（Cache Server）cache，等到需要用的时候再取出来以加速一些运算，在原理上`Cache`的功能类似于其缓存的块（Block）而假如我们把 `Cache` 当作一个 `BasicBlock` 的子类那么就会出现继承的问题：

- 选择继承多个 `BasicBlock` 的子类，就会出现钻石继承的问题
- 增加接口/将所有的 `Block` 做成鸭子类型（但是像 C++ 这样的语言可能不喜欢这样做，虽然可以利用模版实现）


In [14]:
class BasicBlock:
    
    def apply(self):
        pass
    
    def iscacheable(self):
        """
        一个property方法（就不写@property了）
        """
        raise NotImplementedError
        
    def push_cache(self, server):
        """
        上传矩阵
        """
        server.push(self.mat())

## 在Julia里怎么做

Julia的类型只能是鸭子类型，所有的类型只会继承抽象类型的接口（也就是行为），而不是成员。

## 错误的做法

在一些早期Julia代码里，程序是直接从 Python或者 C++ 翻译过来的。我们用这样的风格写一遍上面的代码

In [4]:
# 所有的BasicBlock的子类型都需要有 name 成员
# BasicBlock 的子类下必须有apply方法
# MatrixBlock 的子类必须有mat方法

abstract type BasicBlock end
abstract type MatrixBlock <: BasicBlock end
abstract type CompositeBlock <: MatrixBlock end
abstract type PrimitiveBlock <: MatrixBlock end

如果你这么写了，那么接下来所有的子类下里你都要写一遍 `name` 成员，（而实际上在拥有继承特性的语言里，我们可能不止一个 `name` 成员需要继承）

In [5]:
struct XGate <: PrimitiveBlock
    name::String
end

struct ChainBlock <: CompositeBlock
    name::String
end

这不仅仅很累，还将增加庞大的维护成本：试想如果将来你要删掉，或者修改这个`name` 为 `block_name` 的工作量会有多大？（即便有IDE和智能的编辑器，多了一步也就意味着更多的潜在错误）

回到Julia的文档：我们看看是不是Julia作者脑残不会写 `class`？在 `Types` 这一部分可以看到有这样一段关于继承的描述

> One particularly distinctive feature of Julia's type system is that concrete types may not subtype each other: all concrete types are final and may only have abstract types as their supertypes. While this might at first seem unduly restrictive, it has many beneficial consequences with surprisingly few drawbacks. It turns out that being able to inherit behavior is much more important than being able to inherit structure, and inheriting both causes significant difficulties in traditional object-oriented languages. 

以及在 `Method` 这一部分的描述：

> Multiple dispatch is particularly useful for mathematical code, where it makes little sense to artificially deem the operations to "belong" to one argument more than any of the others: does the addition operation in x + y belong to x any more than it does to y? The implementation of a mathematical operator generally depends on the types of all of its arguments. Even beyond mathematical operations, however, multiple dispatch ends up being a powerful and convenient paradigm for structuring and organizing programs.

那么，两个重点：

- 在Julia里，行为（behavior）的继承更加重要
- 每一种行为的具体实现称为method

这也就意味着，我们要先定义代码要做什么，然后再定义类型。这种设计方式类似于在传统OOP语言里，先定义接口（interface），再开始实现的方式，但依然有不同的地方。

首先，我们所有的块都可以 `apply!`

In [6]:
function apply!(register, blk::BasicBlock)
    # concrete implementation
end

apply! (generic function with 2 methods)

然后对于能够具有矩阵形式的块，`MatrixBlock` 都具有 `mat` 方法

In [7]:
function mat(blk::MatrixBlock)
    # return matrix form
end

mat (generic function with 1 method)

对于这些 `MatrixBlock` 我们可以设置一个默认的 `apply!` 方法（注意之前已经为 `BasicBlock` 定义了一种method）

In [8]:
function apply!(register, blk::MatrixBlock)
    mat(blk) * register
end

apply! (generic function with 3 methods)

其实到这里和传统的OOP写法并无本质区别，但是我们注意原本Python里的block都有一个name成员，这样的类型member共享怎么办？我们在Julia里新建一个 `NamedBlock` 类型

In [9]:
struct NamedBlock{BT <: BasicBlock} <: BasicBlock
    blk::BT
    name::String
end

它的 `apply!` 和 `mat` 方法都会在一定条件下转发:

1. 无条件转发 `apply!` 因为所有的 block 都有这个方法
2. 对成员类型为 `MatrixBlock` 的转发 `mat` 方法 

In [10]:
apply!(register, named_blk::NamedBlock) = apply!(register, named_blk.blk)
mat(register, named_blk::NamedBlock{<:MatrixBlock}) = mat(named_blk.blk)

mat (generic function with 2 methods)

然后接下来定义一些具体实现功能的类型，例如我要计算 `XGate` 那么我们有针对 `XGate` 作用在一个 `GeneralRegister` 上进行优化的 `apply!` 函数，所以我们需要给 `apply!` 函数对 `XGate` 实现一种 method

In [12]:
abstract type GeneralRegister end # 这里只是为了演示方便，定义了一个类型

function apply_x_on_general(r, blk)
    println("applied X on general")
end
apply!(register::GeneralRegister, blk::XGate) = apply_x_on_general(register, blk)

apply! (generic function with 5 methods)

而实际上对于不同的 register 类型，都有特别的优化了的 `apply!` 方法，这个时候你不需要管 `XGate` 继承了谁，这个方法应该属于 register 还是 `XGate`，直接写就好了

In [14]:
abstract type Stabilizer end
abstract type MPS end
abstract type Etc end

apply!(register::Stabilizer, blk::XGate) = 0
apply!(register::MPS, blk::XGate) = 0
apply!(register::Etc, blk::XGate) = 0

apply! (generic function with 8 methods)

耦合低，代码可以在非常独立的 cell 里。适合帮助你专注到复杂的数学计算上。甚至这样的编程方式也适用于其它领域的抽象。

我们在 Yao 里针对不同类型的线路进行优化，很大程度归功于这样的多重派发特性。尽管没有继承，但是通过合理的抽象，我们依然不需要写很多遍重复的类型成员。

### 总结一下

- 先定义（甚至是实现）行为（你到底要算些啥？）
- 再定义类型

问题，类似于上面的 `NamedBlock`，如何增加 `push` 和 `iscacheable` 方法？

## Julia的宏

如果说源代码是产生Julia最上层AST的方式，那么宏则是帮助Julia编译器产生下层AST的助手。

<img src="compile-process-gpu.png" alt="Drawing" style="width: 900px;"/>

那么编写宏有什么注意点呢？

我们回顾一下文档：

> Macros are necessary because they execute when code is parsed, therefore, macros allow the programmer to generate and include fragments of customized code before the full program is run. 

宏更多的是编译时期的一个记号，告诉编译器需要展开这里。而实际上我们仍然需要很多函数去操作 `Expr` 对象，甚至我们在编译时期也是可以应用多重派发来达到一定程度的抽象的（但是这也会增加编译时间）。一般来说，建议将宏的功能转发给函数。

例如这个标准库 `Docs` 里的例子:

`@var` 会将宏接受的表达式转换成一个 `Binding` 对象，供日后给文档系统提供查找的key。

```julia
macro var(x)
    esc(bindingexpr(x))
end
```

它实际上只是在编译时期调用了`bindingexpr`

### Why？

- 更方便的调试

> 我们可以直接利用这些函数测试 `Expr` 对象的输入情况，这个时候因为是运行时的调用，信息更多，方便调试

- 更加模块化，方便和其它宏共享工具

> 例如，我在最近一个叫 `Localize` 的包里想在另外一个宏 `@i18n` 里将表达式转换为 `Binding` 对象，那么我就可以使用这个 `bindingexpr` 函数，而不是再使用一个宏（虽然这也并不是错误的）。